#  Dataset Disponibilizado pela Olist

In [4]:
import pandas as pd

In [3]:
!pip install psycopg2
!pip install sqlalchemy
!pip install pandas
!pip install python-decouple

In [5]:
from sqlalchemy import create_engine
from decouple import config

HOST     = config('HOST')
DATABASE = config('DATABASE')
USER     = config('USER')
PASSWORD = config('PASSWORD')
PORT     = config('PORT')

string_conexao = f'postgresql://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}'
conn_olist = create_engine(string_conexao)

In [6]:
!pip freeze > requirements.txt

#
Formas de inserir valores nas tabelas com o Pandas

In [19]:
df_customers = pd.read_csv("Data/olist_customers_dataset.csv")
df_customers.to_sql("customers", conn_olist, if_exists="replace")

441

In [20]:
df_geolocation = pd.read_csv("Data/olist_geolocation_dataset.csv")
df_geolocation.to_sql("geolocation", conn_olist, if_exists="replace")


163

In [18]:
df_order_items = pd.read_csv("Data/olist_order_items_dataset.csv")
df_order_items.to_sql("order_items", conn_olist, if_exists="replace")

650

In [21]:
df_order_payments = pd.read_csv("Data/olist_order_payments_dataset.csv")
df_order_payments.to_sql("order_payments", conn_olist, if_exists="replace")

886

In [22]:
df_order_reviews = pd.read_csv("Data/olist_order_reviews_dataset.csv")
df_order_reviews.to_sql("order_reviews", conn_olist, if_exists="replace")

224

In [23]:
df_orders = pd.read_csv("Data/olist_orders_dataset.csv")
df_orders.to_sql("orders", conn_olist, if_exists="replace")

441

In [24]:
df_products = pd.read_csv("Data/olist_products_dataset.csv")
df_products.to_sql("products", conn_olist, if_exists="replace")

951

In [25]:
df_sellers = pd.read_csv("Data/olist_sellers_dataset.csv")
df_sellers.to_sql("sellers", conn_olist, if_exists="replace")

95

In [26]:
df_product_category_name_translation = pd.read_csv("Data/product_category_name_translation.csv")
df_product_category_name_translation.to_sql("product_category_name_translation", conn_olist, if_exists="replace")

71